# Clase 21 - Regresión, Ensamblaje y Interpretatividad

## Regresión


Regresión es el problema que consiste en generar modelos que sean capaces de predecir valores reales los cuales son comúnmente llamados target.
Uno de los ejemplos más clásicos (que incluso usamos al comienzo del curso) es, dados los atributos de una vivienda, construir modelos que permitan asignarle un precio de forma automatizada. 

### Regresión Lineal

Sea un conjunto de ejemplos etiquetados (i.e., vectores de $D$ características con sus respectivos targets)
$\{(\mathbf{x}^{(i)}, y^{(i)}) \}_{i=1}^{N}$ respectivamente y con N la cantidad de ejemplos. $x_j^{(i)}$ con $j = 1, \dots, D$ es el vector de características que describe al ejemplo $i$


Una Regresión Lineal consiste en la construcción de un modelo $f_{w,b} = wx + b$ orientado a predecir el target usando una combinación lineal de las características de cada ejemplo de la forma: 

$$f_{\mathbf{w}, b}(x) = \mathbf{wx} + b$$

$$f_{\mathbf{w}, b}(x) = w_{0} x_0 + w_1 x_1 + \dots + w_n x_n$$

En donde $\mathbf{w}$ es el vector de parámetros de tamaño $D$ (que define una pendiente en un hiperplano) y $b$ el intercepto. 
En este caso, se dice que el modelo construido $f_{\mathbf{w}, b}$ está *parametrizado* por $\mathbf{w}$ y $b$.

$f_{\mathbf{w}, b}(x)$ también se le denomina $\hat{y_i}$, el cuál es simplemente es el valor predicho por el modelo.


Cada posible combinación de los parámetros generará una regresión distinta. Por ende, la idea es encontrar el conjunto de parámetros que más se ajusten a los datos 


In [23]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('./resources/auto-mpg.csv')
fig = px.scatter(
    df,
    x='displacement',
    y='mpg',
    template='plotly_white',
    color='mpg',
    hover_name='car name',
    trendline='ols', # este parámetro permite calcular rápidamente la regresión sobre x e y.
    title=
    "Cilindrada de Distintos Automóviles con Respecto a su Rendimiento (mpg)<br>"
    "<sub>La linea representa una regresión lineal calculada sobre ambas variables.</sub>"
)
fig.show()


Para encontrar una buena configuración de parámetros, intentaremos minimizar la siguiente expresión: 


$$\text{Mean Squared Error (MSE)} = \frac{1}{N}\sum_{i=1\dots n} (y_i - f_{\mathbf{w}, b} (x_i))^2 = \frac{1}{N}\sum_{i=1\dots n} (y_i - \hat{y_i})^2$$



En terminos prácticos, MSE es una medida que penaliza que tan mal predice el modelo. Para esto, promedia los cuadrados de la diferencias entre los valores predichos y los valores reales. 

<div align='center'>
<br>
<img src='./resources/mse.png' width=600 />
<br>
    Fuente: <a href='https://vitalflux.com/mean-square-error-r-squared-which-one-to-use/'>Mean Squared Error or R-Squared – Which one to use?</a>.  
</div>
    


En general, las expresiones (como MSE) que buscan maximizar/minimizar valores se les conoce como **función objetivo/función de costo**, mientras que $(f_{\mathbf{w}, b} (x_i) - y_i)^2$ se le denomina **función de pérdida**, la cual se define como el costo/penalización de predecir mal un ejemplo $x_i$.



### Entrenamiento del Modelo

El entrenamiento de los modelos basados en funciones de costo consiste en encontrar los parámetros que permitan reducir al máximo el valor que toma dicha función. En el caso de la regresión lineal, es posible encontrar una expresión analítica (i.e., una fórmula) que permite minimizar la función de costo: Minimos cuadrados.
En este caso, solo veremos el cálculo para una regresión con solo un atributo.

La idea principal detrás de este método es calcular la derivada de MSE sobre cada parámetro ($w$ y $b$) y luego igualar a 0. 

$$\frac{\partial  MSE}{\partial  w} = 0 \leftrightarrow  -2 \sum_{i=1}^{n} (y_i - x_i w - b) x_i = 0$$
$$\frac{\partial  MSE}{\partial b} = 0 \leftrightarrow  -2 \sum_{i=1}^{n} (y_i - x_i w - b) = 0$$


Las ecuaciones anteriores nos permiten calcular $w$ y $b$ un sistema de ecuaciones. Luego de bastantes operaciones algebraicas: 


$$w = \frac{\sum_{i=0}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=0}^{n}(x_i - \bar{x})^2 }$$

$$b = \bar{y} - w\bar{x}$$

Noten que para $w$, la expresión del dividendo es muy similar al cálculo de una correlación entre los atributos y la variable a predecir.
Por otra parte, el divisor es muy similar al cálculo de la varianza de $x$.

In [2]:
# cálculo a mano!

x = df['displacement']
y = df['mpg']

w = sum((x - x.mean()) * (y - y.mean())) / sum((x - x.mean())**2)
b = y.mean() - w * x.mean()

In [3]:
print(f'f(x) = {round(w, 3)}x + {round(b, 3)}')

f(x) = -0.06x + 35.175



## Métricas de Rendimiento de Regresión


### Mean Squared Error 

Una de las principales opciones es utilizar la misma función de costo con la cuál se entrena una Regresión Lineal. Sin embargo, esta cuenta con métricas similares que intentan cuantificar el error de la misma manera:


#### RMSE - Root Mean Squared Error

Esta variante consiste en calcular la raíz cuadrada de MSE. La idea principal de esta métrica es que el error sea interpretable ya que queda con la unidad que se está prediciendo. 

 $$RMSE = \frac{1}{N}\sum_{i=1\dots n} (y_i - \hat{y_i})^2$$
 
 
#### MAE - Mean Absolute Error

Esta variante no calcula el error de forma cuadrática, si no que lo hace simplemente con un valor absoluto

 $$MAE = \frac{1}{N}\sum_{i=1\dots n} |y_i - \hat{y_i}|$$
 
 
#### MAE - Mean Absolute Error

Esta variante no calcula el error de forma cuadrática, si no que lo hace simplemente con un valor absoluto

 $$MAE = \frac{1}{N}\sum_{i=1\dots n} |y_i - \hat{y_i}|$$
 
En este caso, el error también es directamente interpretable: es simplemente el promedio de los errores.


#### MAE - Median Absolute Error

Esta variante, muy similar a la anterior, calcula la mediana en vez de la media de las diferencias.

 $$MedAE = median(|y_1 - \hat{y_1}|, \dots, |y_n - \hat{y_n}|)$$
 
La particularidad de esta formulación es que, a diferencia de la anterior, es resistente a outliers gracias al su cálculo basado en la media.

### Coeficiente de Determinación R²

El Coeficiente de Determinación R² es un puntaje que representa la proporción de varianza de los valores predichos por el modelo con respecto a la los target reales.


$$R^2 (y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y_i})^2}{\sum_{i=1}^n (y_i - \bar{y})^2}$$

En donde $y_i$ es el valor real del target de un ejemplo $x_i$, $\hat{y_i}$ es el valor predicho por el modelo y $\bar{y}$ es la media de los targets reales ($\bar{y} = \frac{1}{n}\sum_{i=1}^{n} y_i$).
$\sum_{i=1}^{n} (y_i - \hat{y_i})^2$ también se conoce como suma de cuadrática de los residuos.


<div align='center'>
<br>
<img src='./resources/r2.png' width=600 />
<br>
<div> 
    Coeficiente de Determinación en <a href='https://en.wikipedia.org/wiki/Coefficient_of_determination'>Wikipedia (Inglés)</a>.
    <br>
En esta infografía, se explica el R² como 1 menos la suma del area los cuadrados azules dividida por la suma de los cuadrados rojos.
    
<br>    
</div>
    
<br>

Puede ser útil pensar que lo que se está comparando es un modelo baseline que siempre predice la media (el de la izquierda) con respecto a un modelo entrenado. Luego, la proporción indica que tanto mejora el modelo entrenado con respecto al baseline. 


El mejor puntaje posible es 1 (los mejores están cercanos a este) y este puede ser negativo (con modelos extremadamente malos).
    
    
---

### Auto-mpg dataset

Para ejemplicar una Regresión Lineal, usaremos el dataset [Auto-mpg dataset](https://www.kaggle.com/uciml/autompg-dataset), el cual, dado distintas características de autos antiguos, intenta predecir el consumo de galones por milla (miles per gallon, mpg)

![Auto-mpg dataset](https://storage.googleapis.com/kaggle-datasets-images/1489/2667/d7895dcd2db5e0cfda19c3edc2f2d410/dataset-cover.jpg)

<div align='center'>
Fuente: Competencia en Kaggle.
</div>


Son 398 autos. Los atributos que los describen son:

    - mpg: continuous
    - cylinders: multi-valued discrete
    - displacement: continuous
    - horsepower: continuous
    - weight: continuous
    - acceleration: continuous
    - model year: multi-valued discrete
    - origin: multi-valued discrete
    - car name: string (unique for each instance)

In [26]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('./resources/auto-mpg.csv')
df = df.dropna()
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [27]:
px.scatter_matrix(df.drop(columns=['car name', 'origin']),
                  title='Scatter Matrix mpg Dataset',
                  height=800,
                  template='plotly_white',
                  color='mpg',
                  color_continuous_scale='Viridis',
                  hover_name='mpg',
                 ).update_traces(diagonal_visible=False, showupperhalf=False)

In [28]:
features = df.drop(columns=['car name', 'mpg'])
target = df['mpg']

features.columns

Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'model year', 'origin'],
      dtype='object')

En el caso de usar todos los atributos, cada observación $\mathbf{x_i}$ estará compuesta por `'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin'` y etiquetada con el target a predecir `mpg` (millas por galón). Al igual que en la clasificación, para entrenar el modelo se separa el dataset en entrenamiento y prueba.



In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
    features, target, shuffle=True, train_size=0.3, random_state=33
)

In [30]:
X_train.head(3)

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
237,4,98.0,63,2051,17.0,77,1
130,4,122.0,80,2451,16.5,74,1
129,4,79.0,67,1950,19.0,74,3


In [31]:
y_train.head(3)

237    30.5
130    26.0
129    31.0
Name: mpg, dtype: float64

In [95]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

preprocessing = ColumnTransformer(
    [
        ("standard", StandardScaler(), ["displacement", "weight", "acceleration"]),
        ("ohe", OneHotEncoder(drop='first'), ["origin"]),
        ("ordinal", OrdinalEncoder(), ["cylinders", "model year"]),
    ]
)

In [96]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

pipe = Pipeline([('preprocesamiento', preprocessing) , ('regresor', LinearRegression())])

In [97]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['displacement', 'weight',
                                                   'acceleration']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first'),
                                                  ['origin']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['cylinders',
                                                   'model year'])])),
                ('regresor', LinearRegression())])

In [98]:
y_pred = pipe.predict(X_test)

In [100]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error

def evaluate(y_test, y_pred):

    print('MSE:', mean_squared_error(y_test, y_pred), '\n')
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
    print('MAE:', mean_absolute_error(y_test, y_pred))
    print('MedAE:', median_absolute_error(y_test, y_pred), '\n')
    print('R²:', r2_score(y_test, y_pred))
    
    
evaluate(y_test, y_pred)

MSE: 12.097738046761647 

RMSE: 3.4781802780709405
MAE: 2.5765475020642516
MedAE: 2.0264467956807444 

R²: 0.8180571551804847


In [103]:
print('Coeficientes de la Regresión (w_i) por Atributo:\n\n', pipe[-1].coef_.round(3))

Coeficientes de la Regresión (w_i) por Atributo:

 [ 0.393 -4.255 -0.123  1.646  1.083 -0.941  0.724]


### Regularización

Regularización es un conjunto de térnicas que fuerzan a la regresión a generar modelos no tan complejos y así evitar el overfitting.

La idea por detrás es simple: para crear modelos regularizados se agregan distintas penalizaciones a la función objetivo (MSE). Estas penalizaciones aumentan el valor de MSE a medida que el modelo se hace más complejo.

Las penalizaciones más comunes consisten en agregar a MSE el cálculo de una norma

- **L2** sobre los parámetros, también conocida como *Rigde*,  
- **L1** sobre los parámetros, también conocida como *Lasso*
- **Elastic-Net**, la cuál es una combinación de las anteriores.




#### Ridge 

$$MSE = \frac{1}{N}\sum_{i=1\dots n} (y_i - f_{\mathbf{w}, b} (x_i))^2  + C ||w||^2$$ 

en donde $||w||$ es la norma se define como $||w||^2 = \sum_{i=1}^D (w_i)^2$ y ||C|| es un hiperparámetro que controla la importancia de la regularización. 

Mientras menor sea $C$, menor efecto tendrá la regularización. Por otra parte, mientras mayor sea, es más posible que la regresión no sea capaz de aprender lo suficientemente bien, lo que puede llevar a un *underfitting*

L2 en general tiende a dar mejores resultados que una regresión normal.

In [112]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

rigde_pipe = Pipeline([('preprocesamiento', preprocessing) , ('regresor', Ridge())])
rigde_pipe.fit(X_train, y_train)

ridge_y_pred = rigde_pipe.predict(X_test)

evaluate(y_test, ridge_y_pred)


MSE: 12.304181382216012 

RMSE: 3.507731657669385
MAE: 2.587015557094784
MedAE: 2.010887541277075 

R²: 0.8149523691782244


#### Lasso

$$MSE = \frac{1}{N}\sum_{i=1\dots n} (y_i - f_{\mathbf{w}, b} (x_i))^2  + \alpha |w|$$ 

en donde $|w|$ es la suma de los valores absolutos de los parámetros y se define como $|w| = \sum_{i=1}^D |w_i|$ y ||C|| es un hiperparámetro que controla la importancia de la regularización. 

Al igual que el caso anterior, mientras menor sea $C$, menor efecto tendrá la regularización. Por otra parte, mientras mayor sea, es más posible que la regresión no sea capaz de aprender lo suficientemente bien, lo que puede llevar a un *underfitting*

Lasso produce modelos sparse, es decir, modelos que muchos de las pendientes de los atributos es igual a 0.
En términos prácticos, produce una selección de atributos al conservar solo los atributos más relevantes para predecir.

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso

lasso_pipe = Pipeline([('preprocesamiento', preprocessing) , ('regresor', Lasso(alpha=0.1))])
lasso_pipe.fit(X_train, y_train)

lasso_y_pred = lasso_pipe.predict(X_test)
evaluate(y_test, lasso_y_pred)

MSE: 12.741019304535723 

RMSE: 3.5694564438490803
MAE: 2.638850281131933
MedAE: 2.0675527355517396 

R²: 0.8083825844792434


In [106]:
print('Coeficientes de la regresión (w_i) usando Lasso por Atributo:\n\n', lasso_pipe[-1].coef_.round(3))

Coeficientes de la regresión (w_i) usando Lasso por Atributo:

 [-0.    -4.102 -0.     0.492  0.    -0.864  0.704]


#### Elastic-Net


$$MSE = \frac{1}{N}\sum_{i=1\dots n} (y_i - f_{\mathbf{w}, b} (x_i))^2  + \alpha \rho |w|$$ 

In [108]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

en_pipe = Pipeline([('preprocesamiento', preprocessing) , ('regresor', ElasticNet(alpha=0.01))])
en_pipe.fit(X_train, y_train)

en_y_pred = en_pipe.predict(X_test)

evaluate(y_test, en_y_pred)

MSE: 12.300509390904685 

RMSE: 3.5072082046700173
MAE: 2.5895442307479177
MedAE: 2.043961850082537 

R²: 0.8150075937617584


### Regresión Logística

Oh, es una clasificación :O

---

## Ensamblaje de Modelos

Es muy probable que para los problemas complejos, los clasificadores vistos anteriormente (KNN, Decision Trees, Bayes) no sean capaces de producir un modelo preciso para resolverlos. 

Si bien el camino usual para resolver el problema de la complejidad es crear modelos de redes neuronales, estos por lo general requieren una gran cantidad de datos para generalizar correctamente. Por ende, si no se posee una mayor cantidad de datos, no siempre será la elección correcta.

Otra opción interesante de analizar es el **ensamblaje de modelos**. 

Los modelos generados por ensamblaje no son otra variedad de modelos más complejos y distintos a los vistos anteriormente, si no que son frameworks para generar **meta-modelos**, es decir, combinaciones de muchos modelos poco complejos y con poca precisión (*weak-learners*).
La hipótesis principal de estos modelos es que la combinación de varios *weak-learners* poco precisos permite generar modelos mucho más complejos con la misma cantidad de datos.


Los *weak learners* son por lo general modelos poco complejos pueden ser entrenados y predecir rápidamente, como por ejemplo los árboles de decisión. La combinación de estos modelos poco complejos se hace a través de algún mecanismo de votación. 

Existen dos paradigmas de creación de modelos ensamblados: **Bagging y Boosting**.

### Bagging (Bootstrap Aggregating)

**Bagging** consiste en entrenar varios *weak learners* (ej, trees) usando distintos muestreos del dataset de entrenamiento y luego combinar los modelos entrenados.


<br>

<div align='center'>
    <img src='./resources/ensemble_bagging.png' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Bootstrap_aggregating'>Boosting en Wikipedia</a>
</div>
<br>



Un ejemplo clásico de este es `RandomForest`.
Su funcionaimento es relativamente sencillo: 

Supongamos que se entrenarán $B$ ($B$ de bags) árboles de decisión y que cada arbol será entrenado con $n$ datos. Entonces,



- Por cada $b=1,...B$:
    - Obtenemos un subconjunto de entrenamiento con n elementos a través de un muestreo con reemplazo (*bootstrap*) sobre conjunto de entrenamiento original.
    - Entrenamos un árbol de decisión para regresión o clasificación $f_b$ usando ese set.

Luego, dado un ejemplo por predecir $x$, para obtener los valores de:

- Una regresión se calcula: 

$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B}f_b(x)$$

- Una clasificación, se retorna la clase con más votos. 


EL siguiente ejemplo consiste en un problema de detección de fraudes bancarios. 
Para mostrar el funcionamiento de Random Forest, evaluaremos la tarea anterior usando un modelo `DecisionTree` y compararemos el aumento de rendimiento usando luego un `RandomForest`

WIP: Descripción del Problema - https://www.kaggle.com/mlg-ulb/creditcardfraud

In [16]:
import pandas as pd

df = pd.read_pickle('./resources/creditcard.pickle')
df = df.drop(columns=['Time'])

In [17]:
# normalise the amount column
features = df.drop(columns=['Class']) 
labels = df['Class']
# as you can see there are 492 fraud transactions.
labels.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    shuffle=True, 
                                                    stratify=labels,
                                                    test_size=0.3,
                                                    random_state=10)

#### Pipeline con DecisionTree

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix

# generar el pipeline
pipe1 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('Decision Tree', DecisionTreeClassifier(random_state=44))
    ])

# entrenar
pipe1.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe1.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Matriz de confusión

[[85256    39]
 [   41   107]]

Reporte de Clasificación

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.73      0.72      0.73       148

    accuracy                           1.00     85443
   macro avg       0.87      0.86      0.86     85443
weighted avg       1.00      1.00      1.00     85443



#### Pipeline Con RandomForest

In [20]:
# generar el pipeline
pipe2 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('Random Forest', RandomForestClassifier(n_jobs=-1, random_state=44))
    ])

# entrenar
pipe2.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe2.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

KeyboardInterrupt: 

Como se puede ver, `RandomForest` incremento en la precisión y recall de la clase 1 con respecto al `DecisionTree`

### Boosting

El segundo enfoque para crear modelos ensamblados es Boosting. En este caso, cada modelo se entrena con todos los datos, pero los modelos se entrenan de forma iterativa de tal forma que los modelos siguientes intentan corregir los errores de los modelos anteriores.

<br>

<div align='center'>
    <img src='./resources/ensemble_boosting.png' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Boosting_(machine_learning)'>Boosting en Wikipedia</a>
</div>
<br>

La idea, en términos simples, es que los primeros modelos predigan bien los casos más generales, mientras que los modelos más profundos se encarguen de predecir los casos más particulares y dificiles.



El modelo ensamblado final será una combinación de los modelos creados en el proceso iterativo. Uno de los algoritmos más populares para crear este tipo de modelos es `GradientBoosting` (usaremos `HistGradientBoostingClassifier`, una implementación mucho más rápida en el ejemplo). Una explicación de su idea principal viene a continuación.


Dada $M$ iteraciones, el algoritmo para generar un modelo para regresión es:

- Comenzamos con un modelo constante $f = f_0$
- Luego, por cada label $i=1,\dots,N$, la transformamos a $\hat{y}_i \leftarrow y_i - f(x_i)$. $\hat{y}_i$ es llamado residual y su conjunto se transforma en las nuevas labels de $x$.
- Ahora, entrenamos un nuevo modelo (tree) $f_1$ y redefinimos $f = f_0 + \alpha f_1$. $\alpha$ es conocido como learning rate.
- Computamos nuevamente los residuales y calculamos un nuevo modelo $f_2$. Luego, fijamos el modelo predictivo como $f = f_0 + \alpha f_1 + \alpha f_2$.
- Repetimos hasta que se cumplan $M$ iteraciones.


La idea general es que en cada iteración los residuales más cercanos a 0 son los que se están clasificando correctamente mientras que los residuales con valores más altos son mal predichos se espera que se vayan corrigiendo en los modelos más profundos hasta alcanzar el modelo $M$.

El caso de la clasificación es un poco más complejo y queda propuesta su explicación.

#### Pipeline con GradientBoosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

# generar el pipeline
pipe3 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('Random Forest', HistGradientBoostingClassifier(
         random_state=44, 
         l2_regularization=10, 
         max_depth=15,
         max_iter=1000))
    ])

# entrenar
pipe3.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe3.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Nuevamente vemos mejores resultados en este tipo de clasificador con respecto al árbol básico. 


En general, Gradient Boosting puede ser más poderoso que los basados en bagging, pero si entrenamiento de caracter secuencial puede hacelo mucho más lento.

**Referencias**


- [1] The Hundred-Page Machine Learning Book, Andriy Burkov. Artículo 7.5

---

## Trabajo con Datasets Desbalanceados (Imbalanced Datasets)

Es muy común que al estar generando modelos para la clasificación, alguna(s) clase(s) del dataset estén desbalanceadas, es decir, tengan muchos menos ejemplos de entrenamiento que otras clases.


Por ejemplo, al clasificar fraudes bancarios, la clase `fraude` probablemente será muy minoritaria respecto a la `no fraude`. 

> **¿Cuándo hay desbalance?**

En general no hay una definición formal de desbalance. La forma más práctica de ver esto es si el desbalance produce errores de clasificación en la(s) clase(s) minoritaria(s). 

Si estamos en un caso de clasificación binaria y tenemos dos clases con 60%/40% de los ejemplos, probablemente este pequeño desbalance no perjudique la clasificación. Por otra parte si tenemos 80%/20 o 90/10%, el desabalance será tal que es posible que el modelo no sea capaz de distinguir bien la clase minoritaria.

> **¿Cuál es la implicancia de esto?**

Un clasificador común entrenará para clasificar la mayor cantidad de ejemplos correctamente. 
Esto implica que la clase mayoritaria tendrá por regla general, un mayor prioridad para ser clasificada de forma correcta que la(s) minoritaria(s).


Consideren la siguiente figura: 
<div align='center'>
<img src='./resources/funcion_decision_cantidad_ejemplos.png' width=600/>
</div>


<div align='center'>
    Fuente: Illustration of the influence of the balancing ratio en <a href='https://imbalanced-learn.org/stable/auto_examples/over-sampling/plot_comparison_over_sampling.html#illustration-of-the-influence-of-the-balancing-ratio'> https://imbalanced-learn.org</a>
</div>

<br>

A medida que se aumenta la cantidad de ejemplos de las clases minoritarias, las fronteras de decisión (*decision boundary*) de cada clase se amplian y se mejoran sus resultados de clasificación.




### Especificar Pesos Por Clase y Por Ejemplo

Una forma de solucionar este problema es utilizar algún clasificador (como SVM) que permita asignarle pesos a las clases de los ejemplos o incluso a los ejemplos mismos de entrenamiento. 

Sin embargo, este es un ejercicio costoso ya que los pesos de cada clase/ejemplo deben ser asignados a mano y implicará una reducción de ejemplos clasificados correctamente en la clase mayoritaria.

Más información de la implementación en scikit en [Unbalanced problems en Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html#unbalanced-problems).


<div align='center'>
<img src='./resources/svm_weighted_class.png'/>
</div>   
    
<div align='center'>
En esta figura se muestra un ejemplo de las decision boundaries de un clasificador que fue entrenado usando pesos personalizados por clase. 
</div>
    
<div align='center'>
<img src='./resources/svm_weighted_examples.png'/>
</div>
En esta figura se muestra un ejemplo de las decision boundaries de un clasificador que fue entrenado usando pesos personalizados por ejemplo. El peso del ejemplo está dado por el tamaño del punto en el scatter. 
<div align='center'>
</div>  

### Oversampling y Undersampling

Otra forma de mejorar los resultados de clasificación de las clases minoritarias es ejecutar Over/Under sampling.

Oversampling consiste en un en aumentar el peso de los ejemplos de entrenamiento a partir de los ejemplos existentes por medio de la repetición de estos. Por otra parte, Undersampling consiste en eliminar aleatoriamente elementos del training set de la clase mayoritaria.

<img src='./resources/over_under_sampling.png'/>

<div align='center'>
    <a href='https://www.kdnuggets.com/2020/01/5-most-useful-techniques-handle-imbalanced-datasets.html'>The 5 Most Useful Techniques to Handle Imbalanced Datasets en KDNuggets</a>
</div>


### Oversampling Sintético

Repetir los ejemplos puede no contribuir en mejorar los decision boundaries. Una forma un poco más inteligente de generar un oversampling es generar ejemplos sintéticos. 

> **¿Cómo podemos hacer ejemplos sintéticos?**

Dos populares algoritmos de esto son *Synthetic Minority Oversampling Technique* (**SMOTE**) y *Adaptive Synthetic* (**ADASYN**).


Dado un ejemplo $x$ de la clase minoritaria, se seleccionan los k vecinos más cercanos y se guardan en el conjunto $S_k$.

Luego, podemos generar nuevos ejemplos sinteticos, primero, tomando algún ejemplo aleatorio $x_{zi} \in S_k$ usando $x_i - \lambda(x_{zi} - x_i)$, en donde $\lambda \in [0,1]$ es un hiperparámetro del método.

Es más sencillo pensar que el ejemplo los puntos que estamos generando son puntos sobre una interpolación entre dos ejemplos existentes:


<div align='center'>
<img src='./resources/smote.png'  width=600/>
</div>

<div align='center'>
    Fuente: <a href='https://www.dropbox.com/s/3l9cp75esgvqpxp/Chapter3.pdf?dl=0'>The Machine Learning Book,  Artículo 3.9: Dealing With Imbalanced Data</a>
</div>


<br>

Luego, un dataset con ejemplos sintéticos generados con SMOTE/ADASYN podría verse como:

<br>

<div align='center'>
<img src='./resources/smote_2.png'  width=600/>
</div>

<br>

<div align='center'>
    Fuente: <a href='https://medium.com/analytics-vidhya/balance-your-data-using-smote-98e4d79fcddb'>The How to Deal with Imbalanced Data using SMOTE</a>
</div>




WIP: Diferencias SMOTE/ADASYN

### SMOTE

In [ ]:
!pip install -U imbalanced-learn
import pandas as pd

WIP: Descripción del Problema

https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
df = pd.read_pickle('./resources/creditcard.pickle')
df = df.drop(columns=['Time'])

In [ ]:
# normalise the amount column
features = df.drop(columns=['Class']) 
labels = df['Class']
# as you can see there are 492 fraud transactions.
labels.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    shuffle=True, 
                                                    stratify=labels,
                                                    test_size=0.3,
                                                    random_state=0)
print('-'*40)
print('Labels de Entrenamiento' )
print(y_train.value_counts())
print('-'*40)
print('Labels de Prueba' )
print(y_test.value_counts())
print('-'*40)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

pipe = Pipeline(
    [('preproc', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('clf', LogisticRegression())
    ])

pipe.fit(X_train, y_train.ravel())
 
y_pred = pipe.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 2,)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
 
print(f'X_train después de SMOTE: {len(X_train_res)}')
print(f'Datos generados: {len(X_train_res) - len(X_train)}','\n')
      
print(f"Ejemplos con Label '1': {sum(y_train_res == 1)}")
print(f"Ejemplos con Label '0': {sum(y_train_res == 0)}")

In [ ]:
pipe.fit(X_train_res, y_train_res.ravel())
 
y_pred = pipe.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

En este caso mejoró mucho el Recall de la clase minoritaria, pero se cometen más errores al clasificar la mayoritaria.

### Undersampling

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours 

enn = EditedNearestNeighbours(n_jobs=-1)

X_train_res, y_train_res = enn.fit_resample(X_train, y_train)
 
print(f'X_train después de NearMiss: {len(X_train_res)}')
print(f'Datos generados: {len(X_train_res) - len(X_train)}','\n')
      
print(f"Ejemplos con Label '1': {sum(y_train_res == 1)}")
print(f"Ejemplos con Label '0': {sum(y_train_res == 0)}")


In [ ]:
pipe.fit(X_train_res, y_train_res.ravel())
 
y_pred = pipe.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Combinación de Ambos

In [ ]:
from imblearn.combine import SMOTEENN 

In [ ]:
sm = SMOTEENN(n_jobs=-1)

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
 
print(f'X_train después de NearMiss: {len(X_train_res)}')
print(f'Datos generados: {len(X_train_res) - len(X_train)}','\n')
      
print(f"Ejemplos con Label '1': {sum(y_train_res == 1)}")
print(f"Ejemplos con Label '0': {sum(y_train_res == 0)}")

In [ ]:
pipe.fit(X_train_res, y_train_res.ravel())
 
y_pred = pipe.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))